In [1]:
import transformers
import pandas as pd
import numpy as np
import torch
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizerFast, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_metric
import pandas as pd

/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-11 16:32:52.941219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 16:32:53.135868: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-11 16:32:53.135929: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not

In [2]:
from transformers import pipeline
#unmasker = pipeline('ner', model='xlm-roberta-base')
#unmasker("Hello I'm a <mask> model.")

In [2]:
list_of_training_data_proc = ["../../gpt2/nov25datafortraining/test_gpt4_498_procedures_from_500_azure.tsv", "../../gpt2/nov25datafortraining/test_gpt4_500_procedures_from_500to1000_azure.tsv", "../../gpt2/nov25datafortraining/test_gpt4_363_procedures_from_1363_azure.tsv"]
list_of_training_data_anamn = ["../../gpt2/nov25datafortraining/test_gpt4_852_anamnesis_from_854_azure.tsv", "../../gpt2/nov25datafortraining/test_gpt4_642_anamnesis_from_645_azure.tsv", "../../gpt2/nov25datafortraining/test_gpt4_465_anamnesis_from_466_azure.tsv", "../../gpt2/nov25datafortraining/test_gpt4_800_anamnesis_from_801_azure.tsv"]

In [3]:
import csv

output_proc = []
output_anamnesis = []

for proc_train in list_of_training_data_proc:
    with open(proc_train, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter="\t")
        output_proc += list(reader)
        
for anamn_train in list_of_training_data_anamn:
    with open(anamn_train, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter="\t")
        output_anamnesis += list(reader)
    
with open('../../gpt2/testset_maria_annotated.tsv', 'r') as csvfile:
    reader2 = csv.reader(csvfile, delimiter="\t")
    output_test = list(reader2)
     

output_proc = [row for row in output_proc]
output_anamnesis = [row for row in output_anamnesis]
output_test = [row for row in output_test]

# labeled_tokens = [(token, label) for token, label in output]

In [6]:
total_anam = []
temporary_anam = []

total_proc = []
temporary_proc = []


for entry in output_proc:
    if len(entry) == 2:
        text = entry[0]
        label = entry[1]
        if label == '0':
            label = 'O'
        if text not in ['SEP', 'CLS']:
            temporary_proc.append([text, label])
    if len(entry) < 2:
        total_proc.append(temporary_proc)
        temporary_proc = []
        
for entry in output_anamnesis:
    if len(entry) == 2:
        text = entry[0]
        label = entry[1]
        if label == '0':
            label = 'O'
        if text not in ['SEP', 'CLS']:
            temporary_anam.append([text, label])
    if len(entry) < 2:
        total_anam.append(temporary_anam)
        temporary_anam = []
        
total_test = []
temporary_test = []

for entry in output_test:
    if len(entry) == 2:
        text = entry[0]
        label = entry[1]
        
        if label in ['0', 'BREAST']:
            label = 'O'
        if text not in ['SEP', 'CLS']:
            temporary_test.append([text, label])
    if len(entry) < 2:
        total_test.append(temporary_test)
        temporary_test = []

In [8]:
# first batch

total = total_anam[:167] + total_proc[:83]
print(len(total))

250


In [9]:
print(len(total))
print(len(total_test))

250
300


In [10]:
def add_chunking(data):
    prev_tag = None
    all_data = []
    for entry in data:
        new_data = []
        for word, tag in entry:
            if tag != 'ADE':
                tag = tag.capitalize()
            if tag == 'O':
                new_data.append((word, tag))
                prev_tag = None
            elif prev_tag == tag:
                new_data.append((word, 'I-' + tag))
            else:
                new_data.append((word, 'B-' + tag))
                prev_tag = tag
        all_data.append(new_data)
    return all_data

In [11]:
print(len(total))
total = add_chunking(total)
print(len(total))
print()
print(len(total_test))
total_test = add_chunking(total_test)
print(len(total_test))

250
250

300
300


In [12]:
labels_ = []
labels_test_ = []

for x in total:
    for y in x:
        label_ = y[1]
        labels_.append(label_)
        
for x in total_test:
    for y in x:
        label_test = y[1]
        labels_test_.append(label_test)

In [13]:
nplabels = np.array(labels_)
nplabels_test = np.array(labels_test_)

In [14]:
unique_values = np.unique(nplabels, return_counts=True)
unique_values_test = np.unique(nplabels_test, return_counts=True)

In [15]:
counts_ = unique_values[1]
names_ = unique_values[0]

counts_test_ = unique_values_test[1]
names_test_ = unique_values_test[0]

print("train:")
for k in range(len(counts_)):
    print(names_[k], "-", counts_[k])

print()
print("test:")
for k in range(len(counts_test_)):
    print(names_test_[k], "-", counts_test_[k])

train:
B-Drug - 273
B-Family - 35
B-Procedure - 778
I-Drug - 118
I-Family - 39
I-Procedure - 994
O - 19168

test:
B-Drug - 347
B-Family - 22
B-Procedure - 572
I-Drug - 309
I-Family - 80
I-Procedure - 612
O - 38959


In [16]:
label_list = list(names_)
label_list_test = list(names_test_)

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

label2id_test = {label: i for i, label in enumerate(label_list_test)}
id2label_test = {i: label for i, label in enumerate(label_list_test)}

In [17]:

print(label_list)
print(label_list_test)

['B-Drug', 'B-Family', 'B-Procedure', 'I-Drug', 'I-Family', 'I-Procedure', 'O']
['B-Drug', 'B-Family', 'B-Procedure', 'I-Drug', 'I-Family', 'I-Procedure', 'O']


In [18]:
# Load the XLM-RoBERTa model and tokenizer
# label2id = {label: i for i, label in enumerate(label_list)}
# id2label = {i: label for i, label in enumerate(label_list)}

model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_list), id2label=id2label, label2id=label2id)
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', max_length=512)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [19]:
val_batch = int((len(total) - int(len(total)*0.85))/2)

In [21]:
anamn_val_batch = total[0:val_batch]
proc_val_batch = total[400:400+val_batch]

train_batch = total[val_batch:400] + total[400+val_batch:]

In [22]:
len(anamn_val_batch + proc_val_batch + train_batch)

250

In [23]:
# split_70 = int(len(all_data) * 0.7)
# split_85 = int(len(all_data) * 0.85)

# training_total = all_data[:split_70]
# validation_total = all_data[split_70:split_85]

validation_total = anamn_val_batch + proc_val_batch
training_total = train_batch
test_total = total_test

In [24]:
print(len(training_total))
print(len(validation_total))
print(len(test_total))

231
19
300


In [30]:
training_total[0]

[('pt', 'O'),
 ('soovib', 'O'),
 ('teha', 'O'),
 ('ct-uuringut', 'B-Procedure'),
 ('rindkerest', 'I-Procedure'),
 (',', 'O'),
 ('kõhust', 'B-Procedure'),
 ('ja', 'O'),
 ('vaagnast', 'B-Procedure'),
 ('.', 'O')]

In [25]:
from datasets import Dataset
import pandas as pd

def get_all_tokens_and_ner_tags(texts):
    return pd.concat([get_tokens_and_ner_tags(texts)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(texts):
    all_tokens = []
    all_entities = []
    for row in texts:
        tokens = [x[0] for x in row]
        entities = [x[1] for x in row]
        
        all_tokens.append(tokens)
        all_entities.append(entities)
    return pd.DataFrame({'tokens': all_tokens, 'ner_tags': all_entities})

def create_sliding_windows(train_dataset, max_length=512, stride=512):
    windowed_dataset = []
    for data in train_dataset:
        tokens = data["tokens"]
        ner_tags = data["ner_tags"]
        for i in range(0, len(tokens), stride):
            window_tokens = tokens[i:i+max_length]
            window_ner_tags = ner_tags[i:i+max_length]
            windowed_data = {"tokens": window_tokens, "ner_tags": window_ner_tags}
            windowed_dataset.append(windowed_data)
    windowss=pd.DataFrame(windowed_dataset)
    return Dataset.from_pandas(windowss)

def get_un_token_dataset(train_directory, test_directory, validation_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    test_df = get_all_tokens_and_ner_tags(test_directory)
    validation_df = get_all_tokens_and_ner_tags(validation_directory)
    
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)
    validation_dataset = Dataset.from_pandas(validation_df)
    
    # Apply sliding window on train_dataset
    train_dataset = create_sliding_windows(train_dataset)
    test_dataset = create_sliding_windows(test_dataset)
    validation_dataset = create_sliding_windows(validation_dataset)
    
    return (train_dataset, test_dataset, validation_dataset)

train_dataset, test_dataset, validation_dataset = get_un_token_dataset(training_total, test_total, validation_total)


In [26]:
print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

231
300
19


In [33]:
train_dataset[0]

{'tokens': ['pt',
  'soovib',
  'teha',
  'ct-uuringut',
  'rindkerest',
  ',',
  'kõhust',
  'ja',
  'vaagnast',
  '.'],
 'ner_tags': ['O',
  'O',
  'O',
  'B-Procedure',
  'I-Procedure',
  'O',
  'B-Procedure',
  'O',
  'B-Procedure',
  'O']}

In [27]:
print(train_dataset[0])
print()
print(validation_dataset[0])
print()
print(test_dataset[0])

{'tokens': ['NAME', ':', 'DATE.', 'rinna', 'op'], 'ner_tags': ['O', 'O', 'O', 'B-Procedure', 'I-Procedure']}

{'tokens': ['kiiritusravi', 'planeerimine', ',', 'ravi', 'selgitatud', ',', 'patsient', 'raviga', 'nõus', ',', 'allkirjastab', 'nõusoleku.', 'DATE.', 'alustab', 'raviga', ',', 'nahahooldus.kiiritusravi', 'talutava', 'probleemideta', ',', 'jätkab', 'samal', 'viisil.', 'DATE.', 'lõpetab', 'raviga'], 'ner_tags': ['B-Procedure', 'I-Procedure', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Procedure', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}

{'tokens': ['Perearst', 'saadab', 'õhupuudusega', ',', 'teadmata', 'ajast', 'FA', 'teke', ',', 'III', 'korruse', 'minnes', 'õhupuudus.', 'Stenokarilisi', 'kaebusi', 'otseselt', 'ei', 'ole.põdes', 'astmat', '2', 'a.-t', 'Lõpetas', 'suitsetamise', '4a.-t', 'tagasi.Allergia', 'taimedele', ',', 'anamneesi', 'annab', 'rütmihäirete', 'osas', 'juba', '20', 'aastat.', 'Alustame', 'marevaniseerimist.', 'INR', 'hoida', 'pe

In [28]:
print(label2id)
print(label2id_test)

print(id2label)
print(id2label_test)

{'B-Drug': 0, 'B-Family': 1, 'B-Procedure': 2, 'I-Drug': 3, 'I-Family': 4, 'I-Procedure': 5, 'O': 6}
{'B-Drug': 0, 'B-Family': 1, 'B-Procedure': 2, 'I-Drug': 3, 'I-Family': 4, 'I-Procedure': 5, 'O': 6}
{0: 'B-Drug', 1: 'B-Family', 2: 'B-Procedure', 3: 'I-Drug', 4: 'I-Family', 5: 'I-Procedure', 6: 'O'}
{0: 'B-Drug', 1: 'B-Family', 2: 'B-Procedure', 3: 'I-Drug', 4: 'I-Family', 5: 'I-Procedure', 6: 'O'}


In [29]:
task = "ner"

def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print(label2id)

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)
validation_tokenized_datasets = validation_dataset.map(tokenize_and_align_labels, batched=True)

{'B-Drug': 0, 'B-Family': 1, 'B-Procedure': 2, 'I-Drug': 3, 'I-Family': 4, 'I-Procedure': 5, 'O': 6}


In [30]:
batch_size = 16

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01
)

data_collator = DataCollatorForTokenClassification(tokenizer)


metric = load_metric("seqeval")
results_collection = []


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    

    results = metric.compute(predictions=true_predictions, references=true_labels)
    results_collection.append(results)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    


trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=validation_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('xlmr_gpt4_batch05_250.model')

/tmp/ipykernel_35691/2251306310.py:16: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The following columns in the training set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 231
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 120
  Number of trainable parameters = 277458439
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-12-11 16:36:30.356160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 16:36:30.527830: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-11 16:36:30.527869: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.663505,0.000000,0.000000,0.000000,0.838393
2,No log,0.623420,0.000000,0.000000,0.000000,0.838393
3,No log,0.592327,0.000000,0.000000,0.000000,0.838393
4,No log,0.559209,0.303030,0.042017,0.073801,0.842857
5,No log,0.527341,0.287879,0.079832,0.125000,0.847768
6,No log,0.515644,0.532710,0.239496,0.330435,0.861607
7,No log,0.495981,0.511278,0.285714,0.366577,0.860268
8,No log,0.500572,0.472393,0.323529,0.384040,0.861161


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 19
  Batch size = 16
/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/space/home/shuva/miniconda3/envs/roberta/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

Saving model checkpoint to xlmr_gpt4_batch05_250.model
Configuration saved in xlmr_gpt4_batch05_250.model/config.json
Model weights saved in xlmr_gpt4_batch05_250.model/pytorch_model.bin
tokenizer config file saved in xlmr_gpt4_batch05_250.model/tokenizer_config.json
Special tokens file saved in xlmr_gpt4_batch05_250.model/special_tokens_map.json


In [31]:
predictions, labels, _ = trainer.predict(test_tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list_test[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 300
  Batch size = 16


{'Drug': {'precision': 0.5543318649045521,
  'recall': 0.6328583403185247,
  'f1': 0.5909980430528375,
  'number': 1193},
 'Family': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 56},
 'Procedure': {'precision': 0.2830687830687831,
  'recall': 0.1096311475409836,
  'f1': 0.15805022156573115,
  'number': 1952},
 'overall_precision': 0.45750708215297453,
 'overall_recall': 0.302717900656045,
 'overall_f1': 0.3643542019176537,
 'overall_accuracy': 0.93797033950239}

In [32]:
from seqeval.metrics import classification_report

print(classification_report(true_labels, true_predictions))

              precision    recall  f1-score   support

        Drug       0.55      0.63      0.59      1193
      Family       0.00      0.00      0.00        56
   Procedure       0.28      0.11      0.16      1952

   micro avg       0.46      0.30      0.36      3201
   macro avg       0.28      0.25      0.25      3201
weighted avg       0.38      0.30      0.32      3201

